In [ ]:
def estadistica_descriptiva_imagen(inp01,b=8):
  return np.mean(inp01), np.std(inp01), np.uint32(np.min(inp01)), np.uint32(np.max(inp01)), 100*((inp01.max() - inp01.min())/((2**b)-1))
def histograma(inp01, b=8, mostrar=True):
  intensidades = np.zeros(2**b)
  for k in range(2**b):
    intensidades[k] = np.count_nonzero(inp01 == k)

  if mostrar:
    fig = plt.figure(figsize=(12,8))
    plt.plot(intensidades)
    plt.xlabel("Intensidades")
    plt.ylabel("Frecuencia")
    plt.title("Histograma")
    plt.grid()
    plt.show()
  return intensidades, np.array(list(range(2**b)))